Importando librerias

In [8]:
import numpy as np
import ipyparallel as ipp
%matplotlib inline

Mostrando los clientes cluster

In [10]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [11]:
c[1:3].apply_sync(lambda: "Hola clase de MCA")

['Hola clase de MCA', 'Hola clase de MCA']

apply_sync aplica la función a todos los motores

In [12]:
c[0].apply_sync(lambda x,y,z: x+y+z,range(10), range(10), range(10))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9]

map_sync distribuye la función entre todos los motores y la aplica

In [13]:
dview = c[:]

In [14]:
c[:].map_sync(lambda x,y,z: x+y+z,range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Creamos una función PrimeQ0 optimizando la creación de lista de números con np.arange()

In [15]:
def PrimeQ0(x):
    import numpy as np
    return all(x % i for i in np.arange(int(x**0.5)+1)[2:])

In [191]:
PrimeQ0(2**7-1)

True

In [17]:
PrimeQ=lambda x: all(x % i != 0 for i in np.arange(int(x**0.5)+1)[2:]) and x>1

In [18]:
c[:].apply_sync(PrimeQ0, 2**7)

[False, False, False, False]

In [19]:
dview.apply_sync(PrimeQ0, 2**7)

[False, False, False, False]

In [20]:
2**np.arange(7)

array([ 1,  2,  4,  8, 16, 32, 64])

In [21]:
%timeit dview.map_sync(PrimeQ0, 2**np.arange(25))

10 loops, best of 3: 25.6 ms per loop


In [22]:
%%timeit
for i in 2**np.arange(25):
    x % i

NameError: global name 'x' is not defined

In [23]:
def PrimeQ1(x):
    import numpy as np
    return np.all(np.array([x % i for i in np.arange(int(x**0.5)+1)[2:]]))

In [24]:
%timeit dview.map_sync(PrimeQ1, 2**np.arange(25))

10 loops, best of 3: 27.1 ms per loop


In [25]:
np.mod?

In [26]:
19 % 12

7

In [27]:
np.mod(12,24)

12

In [28]:
np.arange(24)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [29]:
np.mod(np.arange(24),12)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11])

In [67]:
np.mod(12, np.arange(24)[2:])

array([ 0,  0,  0,  2,  0,  5,  4,  3,  2,  1,  0, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12])

In [63]:
np.array([hora % 12 for hora in np.arange(24)])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11])

In [60]:
12 % 12

0

In [30]:
for med in np.arange(12,24):
    #print med
    print np.mod(np.arange(24), med)

[ 0  1  2  3  4  5  6  7  8  9 10 11  0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12  0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13  0  1  2  3  4  5  6  7  8  9]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0  1  2  3  4  5  6  7]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16  0  1  2  3  4  5  6]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17  0  1  2  3  4  5]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18  0  1  2  3  4]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19  0  1  2  3]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20  0  1  2]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21  0  1]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22  0]


In [208]:
def PrimeQ2(x):
    primos = []
    import numpy as np
    lista = np.arange(np.int(x**0.5)+1)[2:]
    residuos = np.mod(x, lista)
    if (np.all(residuos)):
        return x
    else:
        return 0

In [228]:
lista = dview.map_sync(PrimeQ2, 2**np.arange(10)-1)

In [229]:
print lista

[0, 1, 3, 7, 0, 31, 0, 127, 0, 0]
